## Chinese Loans to African Countries by Natural Resource Reserves

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import countries_listed

/Users/jainsabr/workspace/china_africa_aid_resources/env/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


## Import Data

In [ ]:
chinese_dev_finance = pd.read_excel(r'AidData_GCDFD_v2.xlsx', sheet_name='AidData_GCDFD_v2')
rare_earths = pd.read_csv(r'./ree/main.csv')

In [ ]:
# group by country
rare_earth_country = rare_earths.groupby(['country']).size().reset_index()

# Filter out certain flow types
chinese_dev_finance_grants = chinese_dev_finance[chinese_dev_finance['Flow Type'] == 'Grant']
chinese_dev_finance_loans = chinese_dev_finance[chinese_dev_finance['Flow Type'] == 'Loan']

# TO-DO : Reconcile country names (some are different in both data sets)

In [ ]:
# Clean duplicate + regional names
names_to_be_replaced = {'Viet Nam': 'Vietnam', 'Congo (Zaire)': 'Congo', 'Surinam': 'Suriname', 'Burma': 'Myanmar', 'Kyrgyz Republic': 'Kyrgyzstan', "Democratic People's Republic of Korea": 'North Korea'}
for old_name, new_name in names_to_be_replaced.items():
    chinese_dev_finance['Recipient'] = chinese_dev_finance['Recipient'].replace([old_name],new_name)
    rare_earth_country['country'] = rare_earth_country['country'].replace([old_name],new_name)

names_to_delete = ['Asia, regional', 'Oceania, regional', 'Europe, regional', 'America, regional', 'Middle East, regional']
for name in names_to_delete:
    chinese_dev_finance = chinese_dev_finance[chinese_dev_finance['Recipient'] != name]

x = chinese_dev_finance['Recipient'].tolist()
y = rare_earth_country['country'].tolist()
print(list(set(x) - set(y))) # in China finance dataset but not in rare earths dataset
print(list(set(y) - set(x))) # in rare earths dataset but not in China dataset

# Run OLS

In [ ]:
def combine_dataframes (cfd_df, rare_earth_df, sep_year=False, cfd_dropna=True, rare_earth_dropna=False): 
    cfd_df = cfd_df[['Recipient', 'Commitment Year', 'Amount (Constant USD2017)']]
    if not sep_year:
        cfd_df = cfd_df.groupby(['Recipient'])['Amount (Constant USD2017)'].sum().reset_index()
    else:
        cfd_df = cfd_df.groupby(['Recipient', 'Commitment Year'])['Amount (Constant USD2017)'].sum().reset_index()
    rare_earth_df = rare_earth_df.groupby(['country']).size().reset_index()

    # rename columns
    cfd_df = cfd_df.rename(columns={'Recipient':'country'})
    rare_earth_df = rare_earth_df.rename(columns={0: 'Rare Earth Deposits'})

    # deal with null values
    if cfd_dropna and rare_earth_dropna:
        result = pd.merge(cfd_df, rare_earth_df, on='country')
    elif cfd_dropna:
        result = pd.merge(cfd_df, rare_earth_df, on='country', how='left')
    elif rare_earth_dropna:
        result = pd.merge(cfd_df, rare_earth_df, on='country', how='right')
    else:
        result = pd.merge(cfd_df, rare_earth_df, on='country', how='outer')
    result = result.fillna(0)

    return result

In [ ]:
def run_ols (df, x_vars='Rare Earth Deposits', y_vars='Amount (Constant USD2017)', constant=True):
    x = df[x_vars].tolist()
    y = df[y_vars].tolist()

    y = np.log(y)

    if constant:
        x = sm.add_constant(x)

    # performing the regression
    # and fitting the model
    result = sm.OLS(y, x).fit()
  
    # printing the summary table
    print(result.summary())

In [ ]:
# all recipients, all flows
combined_agg = combine_dataframes(chinese_dev_finance, rare_earths)
run_ols(combined_agg)

In [ ]:
# all countries w/ rare earths, all flows
combined_agg_allcountries = combine_dataframes(chinese_dev_finance, rare_earths, cfd_dropna=False)
run_ols(combined_agg_allcountries)

In [ ]:
# all recipients, only loans
combined_onlyloans = combine_dataframes(chinese_dev_finance_loans, rare_earths)
run_ols(combined_onlyloans)

In [ ]:
# all recipients, only grants
combined_onlygrants = combine_dataframes(chinese_dev_finance_grants, rare_earths)
run_ols(combined_onlygrants)

In [ ]:
# all recipients w/rare earths
combined_recip_with_re = combine_dataframes(chinese_dev_finance, rare_earths, rare_earth_dropna=True)
run_ols(combined_recip_with_re)